<a href="https://colab.research.google.com/github/guiraposo/GRpy/blob/main/TOV_in_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sympy

In [20]:
import sympy as sp

In [155]:
t, theta, phi= sp.symbols('t theta phi ')
r, M = sp.symbols('r M', positive=True)

alpha = sp.Function('alpha')(r)
m = sp.Function('m')(r)
rho = sp.Function('rho')(r)  # Energy density
p = sp.Function('p')(r)      # Pressure
g = sp.Matrix([[- sp.exp(2*alpha), 0, 0, 0],
               [0, (1- 2*m/r)**(-1), 0, 0],
               [0, 0, r**2, 0],
               [0, 0, 0, r**2 * sp.sin(theta)**2]])

u_t = -sp.sqrt(-g[0, 0])  # Time component of the four-velocity, assuming normalization
u = [u_t, 0, 0, 0]  # Four-velocity components in static spacetime


In [150]:
def christoffel_symbols(metric, coords):
    dims = metric.shape[0]
    christoffel = [[[0 for _ in range(dims)] for _ in range(dims)] for _ in range(dims)]
    inv_metric = metric.inv()
    for k in range(dims):
        for i in range(dims):
            for j in range(dims):
                sum_term = 0
                for l in range(dims):
                    partial_1 = sp.simplify(sp.diff(metric[j, l], coords[i]))
                    partial_2 = sp.simplify(sp.diff(metric[i, l], coords[j]))
                    partial_3 = sp.simplify(sp.diff(metric[i, j], coords[l]))
                    sum_term += inv_metric[k, l] * (partial_1 + partial_2 - partial_3)
                christoffel[k][i][j] = sp.simplify(sum_term / 2)
    return christoffel
def riemann_tensor(christoffel, coords):
    dims = len(coords)
    R = [[[sp.zeros(dims, dims) for _ in range(dims)] for _ in range(dims)] for _ in range(dims)]
    for rho in range(dims):
        for sigma in range(dims):
            for mu in range(dims):
                for nu in range(dims):
                    term1 = sp.simplify(sp.diff(christoffel[rho][nu][sigma], coords[mu]))
                    term2 = sp.simplify(sp.diff(christoffel[rho][mu][sigma], coords[nu]))
                    term3 = sp.simplify(sum(christoffel[rho][mu][lambda1] * christoffel[lambda1][nu][sigma] for lambda1 in range(dims)))
                    term4 = sp.simplify(sum(christoffel[rho][nu][lambda1] * christoffel[lambda1][mu][sigma] for lambda1 in range(dims)))
                    R[rho][sigma][mu][nu] = sp.simplify(term1 - term2 + term3 - term4)
    return R
def ricci_tensor(riemann, coords):
    dims = len(coords)
    ricci = sp.zeros(dims, dims)
    for mu in range(dims):
        for nu in range(dims):
            ricci[mu, nu] = sp.simplify(sum(riemann[lambda1][mu][lambda1][nu] for lambda1 in range(dims)))
    return ricci
def ricci_scalar(ricci, metric):
    inv_metric = metric.inv()
    dims = metric.shape[0]
    scalar = 0
    for mu in range(dims):
        for nu in range(dims):
            scalar += sp.simplify(inv_metric[mu, nu] * ricci[mu, nu])
    return sp.simplify(scalar)
def einstein_tensor(ricci_tensor, ricci_scalar, metric):
    dims = metric.shape[0]
    einstein = sp.zeros(dims, dims)
    for mu in range(dims):
        for nu in range(dims):
            einstein[mu, nu] = ricci_tensor[mu, nu] - 1/2 * metric[mu, nu] * ricci_scalar
    return sp.simplify(einstein)
def stress_energy_tensor_perfect_fluid(rho, p, u, metric):
    dims = metric.shape[0]
    stress_energy = sp.zeros(dims, dims)
    for mu in range(dims):
        for nu in range(dims):
            stress_energy[mu, nu] = (rho + p) * u[mu] * u[nu] + p * metric[mu, nu]
    return sp.simplify(stress_energy)
def einsteins_equations(einstein_tensor, stress_energy_tensor_perfect_fluid, metric):
    dims = metric.shape[0]
    einstein_eqs = sp.zeros(dims, dims)
    for mu in range(dims):
        for nu in range(dims):
            einstein_eqs[mu, nu] = einstein_tensor[mu, nu] - 8 * sp.pi * stress_energy_tensor_perfect_fluid[mu,nu]
    # Einstein's Equations: G_{\mu\nu} - 8πT_{\mu\nu} = 0 (In geometric units)
    # We return a tensor that should be all zeros if the equations are satisfied
    return sp.simplify(einstein_eqs)


In [156]:
coords = [t, r, theta, phi]  # Assuming these are your coordinates
christoffel_symbols_g = christoffel_symbols(g, coords)
riemann_tensor_g = riemann_tensor(christoffel_symbols_g, coords)
ricci_tensor_g = ricci_tensor(riemann_tensor_g, coords)
ricci_scalar_g = ricci_scalar(ricci_tensor_g, g)
einstein_tensor_g = einstein_tensor(ricci_tensor_g, ricci_scalar_g, g)
Tdd_g = stress_energy_tensor_perfect_fluid(rho, p, u, g)
einstein_eqs_g = einsteins_equations(einstein_tensor_g, Tdd_g, g)

In [157]:
alpha_test = 1/2 * sp.log(1 - 2*M / r)
m_test = M
rho_test = 0
p_test = 0
christoffel_symbols_substituted = [[[
    sp.simplify(symbol.subs({alpha: alpha_test, m: m_test}))
    for symbol in row]
    for row in matrix]
    for matrix in christoffel_symbols_g]


In [158]:
ricci_scalar_g

-2*Derivative(alpha(r), r)**2 - 2*Derivative(alpha(r), (r, 2)) + 4*m(r)*Derivative(alpha(r), r)**2/r + 4*m(r)*Derivative(alpha(r), (r, 2))/r + 2*Derivative(alpha(r), r)*Derivative(m(r), r)/r - 4*Derivative(alpha(r), r)/r + 6*m(r)*Derivative(alpha(r), r)/r**2 + 4*Derivative(m(r), r)/r**2

In [159]:
sp.simplify(sp.simplify(ricci_scalar_g.subs({alpha: alpha_test, m: m_test})))

0

In [160]:
sp.simplify(sp.simplify(einstein_tensor_g.subs({alpha: alpha_test, m: m_test})))

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

In [135]:
Tdd_g

Matrix([
[rho*A(r)**2,         0,      0,                    0],
[          0, p*B(r)**2,      0,                    0],
[          0,         0, p*r**2,                    0],
[          0,         0,      0, p*r**2*sin(theta)**2]])

In [161]:
einstein_eqs_g

Matrix([
[(-8*pi*r**2*rho(r) + 2.0*Derivative(m(r), r))*exp(2*alpha(r))/r**2,                                                                                                                                0,                                                                                                                                                                                                                                                                                                                                             0,                                                                                                                                                                                                                                                                                                                                                                                                                    0],
[                                                 

In [142]:
sp.simplify(sp.simplify(einstein_eqs_g.subs({A: A_test, B: B_test, rho: rho_test, p: p_test})))

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])